In [14]:
import os
import pickle
import argparse

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor  # Change: Using ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder

In [2]:
#define the training filepath
filename_sept_test = './dataset/yellow_tripdata_2023-09.parquet'
model_output_filepath = './models'

In [ ]:
def data_preprocess(filename):
    # Check file format and read DataFrame accordingly
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        
        # Convert datetime columns to pandas datetime objects
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
        
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    # Calculate trip duration in minutes
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    # Filter trips based on duration (between 1 and 60 minutes)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # Convert selected columns to string type for categorical representation
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    # Create a new column 'PU_DO' by combining pickup and dropoff location IDs
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    # Return the processed DataFrame
    return df


In [ ]:
# Preprocess the test data
df_test = data_preprocess(filename_sept_test).head(1)

In [ ]:
# Define categorical and numerical features : independant features
categorical = ['PU_DO']
numerical = ['trip_distance']

# Initialize a DictVectorizer
dv = DictVectorizer()

# Convert test data to a dictionary of records and then transform it into a sparse matrix
test_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(test_dicts)

#dependant feature/ target variable

target = 'duration'

y_test = df_test[target].values

In [20]:
# Define the input features
PU_DU = '163_230'
trip_distance = 0.80

# Assuming 'PU_DU' contains two numerical values separated by an underscore
PU_DU_numeric = np.array(list(map(int, PU_DU.split('_')))).reshape(1, -1)


In [21]:
# Combine 'PU_DU_encoded' with 'trip_distance'
new_data_point = np.concatenate([PU_DU_numeric, np.array([[trip_distance]])], axis=1)


In [17]:
# Load the Extra Trees model
model_path = './models/extra_trees_model.pkl'
with open(model_path, 'rb') as file:
    extra_trees_model = pickle.load(file)

In [22]:
# Make predictions using the loaded model
prediction = extra_trees_model.predict(new_data_point)

ValueError: X has 3 features, but ExtraTreesRegressor is expecting 714 features as input.

In [ ]:
# Print or use the prediction as needed
print(prediction)

In [ ]:
def predict(location1, location2):
    # load model from bin folder
    model = '.models/extra_trees_model.pkl'

    #concat-

    # concat location1, location2
    # predict using the model for the given location1,location2
    # return predicted time 
    
    
